In [16]:
import pandas as pd
import pickle

In [2]:
library_directory = "library.csv"
library_text = "library.parquet"
library_vectors = "library.pkl"

In [22]:
def get_directory(file_path="library.csv"):
    return pd.read_csv(file_path)

def get_text(file_path):
    return pd.read_parquet(file_path)

def get_vectors(file_path="library.pkl"):
    with open(file_path, "rb") as f:
        return pickle.load(f)

def load_library(csv_path="library.csv",
                 parquet_path="library.parquet",
                 pkl_path="libary.pkl"):
    
    metadata = get_directory(csv_path)
    text = get_text(parquet_path)
    vectors = get_vectors(pkl_path)

    return metadata, text, vectors

In [24]:
metadata, text, vectors = load_library(library_directory,
                                      library_text,
                                      library_vectors
                                     )

In [25]:
metadata

,chunk_uid,document,document_type,n_tokens,embedding_model,character_start,character_end,timestamp_iso,timestamp_ms
0,483c073d837ac8e09f4730de2c07ff0e66d05c28867abe...,Obfuscation,.docx,207,sentence-transformers/all-MiniLM-L6-v2,0,1049,2025-09-01T17:51:24.989Z,1756749084989
1,2791d212bef0adec6dc00583aec2aab49dc787537f62e3...,Obfuscation,.docx,239,sentence-transformers/all-MiniLM-L6-v2,591,1846,2025-09-01T17:51:25.029Z,1756749085029
2,2173cf50e5da055b5ff7af6204187280a9837cc0a33b3a...,Obfuscation,.docx,201,sentence-transformers/all-MiniLM-L6-v2,1317,2456,2025-09-01T17:51:25.070Z,1756749085070
3,42596d47845c415b3d41617c91ec8bb43f18d7707d447b...,Obfuscation,.docx,193,sentence-transformers/all-MiniLM-L6-v2,2174,3268,2025-09-01T17:51:25.113Z,1756749085113
4,9534d4fcf385055a20d14e5525977634c2de6adb0b633b...,Obfuscation,.docx,234,sentence-transformers/all-MiniLM-L6-v2,2809,4061,2025-09-01T17:51:25.154Z,1756749085154
...,...,...,...,...,...,...,...,...,...
229,ba4fb62b259d449e47546eb71d158fada9d4a3df463672...,2001.00973v1,.pdf,234,sentence-transformers/all-MiniLM-L6-v2,816,1708,2025-09-01T18:14:21.897Z,1756750461897
230,041808f36e2a9383202d87bb8ebb7dcc13730cb2c3cb12...,2001.00973v1,.pdf,236,sentence-transformers/all-MiniLM-L6-v2,1523,2389,2025-09-01T18:14:21.965Z,1756750461965
231,9f862199f238cfa23e944fe4a20c6764993bd7d03132b2...,2001.00973v1,.pdf,229,sentence-transformers/all-MiniLM-L6-v2,2182,3164,2025-09-01T18:14:22.028Z,1756750462028
232,509234213622339577397af55e0610710c9a502bdffa99...,2001.00973v1,.pdf,224,sentence-transformers/all-MiniLM-L6-v2,2904,3827,2025-09-01T18:14:22.089Z,1756750462089


In [26]:
text

,chunk_uid,text
0,483c073d837ac8e09f4730de2c07ff0e66d05c28867abe...,1. Introduction: The problem of data gathering...
1,2791d212bef0adec6dc00583aec2aab49dc787537f62e3...,"So far, so normal — but the appearance of “loy..."
2,2173cf50e5da055b5ff7af6204187280a9837cc0a33b3a...,Whether because they resented the lack of choi...
3,42596d47845c415b3d41617c91ec8bb43f18d7707d447b...,Innocuous traces of everyday life submitted to...
4,9534d4fcf385055a20d14e5525977634c2de6adb0b633b...,"Linking contemporary and historical examples, ..."
...,...,...
229,ba4fb62b259d449e47546eb71d158fada9d4a3df463672...,Commercial aviation safety 5/e. McGraw Hill Pr...
230,041808f36e2a9383202d87bb8ebb7dcc13730cb2c3cb12...,"Journal of Business Ethics 64, 3 (2006), 271–2..."
231,9f862199f238cfa23e944fe4a20c6764993bd7d03132b2...,Algorithmic accountability. Philosophical Tran...
232,509234213622339577397af55e0610710c9a502bdffa99...,Edward Elgar Publishing. [75] Alexander Styhre...


In [36]:
print(len(vectors))
vectors[0]

234


{'chunk_uid': '483c073d837ac8e09f4730de2c07ff0e66d05c28867abe4ac29716f57fe00e51',
 'vector_embedding': array([-8.09205025e-02,  6.46898225e-02, -2.70382315e-02,  1.00401826e-02,
        -1.51437204e-02,  3.09876967e-02,  3.85005698e-02,  2.67214840e-03,
         2.77516246e-02, -7.39780590e-02,  1.05961837e-01,  9.21992511e-02,
         4.17809896e-02, -6.89194202e-02,  4.22008149e-02, -1.72236152e-02,
         2.34426577e-02, -1.88193412e-03, -9.90582928e-02,  4.91645699e-03,
        -7.21929297e-02, -4.46846113e-02, -3.09186112e-02,  9.57297832e-02,
        -5.78068197e-02,  1.23801548e-02,  1.26326019e-02, -7.11230338e-02,
        -5.10372035e-02, -5.49402870e-02,  8.76601487e-02,  3.12060472e-02,
         1.05212063e-01,  5.83606735e-02,  2.11916342e-02, -8.95082355e-02,
         5.31721935e-02, -1.61289331e-02, -2.06716098e-02, -3.60571705e-02,
         2.72286348e-02, -9.12689567e-02, -1.16106279e-01,  6.17461763e-02,
         1.62455011e-02,  2.17806175e-02,  5.00700623e-02,  6.